# ML in Cybersecurity: Task 3

## Team
  * **Team name**:  *fill this in*
  * **Members**:  *fill this in. format: name1 (email1), name2 (email2), name3 (email3),*


## Logistics
  * **Due date**: 9th December 2021, 23:59:59
  * Email the completed notebook to: `mlcysec_ws2022_staff@lists.cispa.saarland`
  * Complete this in **teams of 3**
  * Feel free to use the forum to discuss.
  
## Timeline
  * 26-Nov-2021: hand-out
  * **09-Dec-2021**: Email completed notebook
  
  
## About this Project
In this project, you will explore an application of ML to a popular task in cybersecurity: malware classification.
You will be presented with precomputed behaviour analysis reports of thousands of program binaries, many of which are malwares.
Your goal is to train a malware detector using this behavioural reports.


## A Note on Grading
The grading for this project will depend on:
 1. Vectorizing Inputs
   * Obtaining a reasonable vectorized representations of the input data (a file containing a sequence of system calls)
   * Understanding the influence these representations have on your model
 1. Classification Model  
   * Following a clear ML pipeline
   * Obtaining reasonable performances (>60\%) on held-out test set
   * Choice of evaluation metric
   * Visualizing loss/accuracy curves
 1. Analysis
   * Which methods (input representations/ML models) work better than the rest and why?
   * Which hyper-parameters and design-choices were important in each of your methods?
   * Quantifying influence of these hyper-parameters on loss and/or validation accuracies
   * Trade-offs between methods, hyper-parameters, design-choices
   * Anything else you find interesting (this part is open-ended)


## Grading Details
 * 40 points: Vectorizing input data (each input = behaviour analysis file in our case)
 * 40 points: Training a classification model
 * 15 points: Analysis/Discussion
 * 5 points: Clean code
 
## Filling-in the Notebook
You'll be submitting this very notebook that is filled-in with your code and analysis. Make sure you submit one that has been previously executed in-order. (So that results/graphs are already visible upon opening it). 

The notebook you submit **should compile** (or should be self-contained and sufficiently commented). Check tutorial 1 on how to set up the Python3 environment.


**The notebook is your project report. So, to make the report readable, omit code for techniques/models/things that did not work. You can use the final summary to provide a report about these.**

It is extremely important that you **do not** re-order the existing sections. Apart from that, the code blocks that you need to fill-in are given by:
```
#
#
# ------- Your Code -------
#
#
```
Feel free to break this into multiple-cells. It's even better if you interleave explanations and code-blocks so that the entire notebook forms a readable "story".


## Code of Honor
We encourage discussing ideas and concepts with other students to help you learn and better understand the course content. However, the work you submit and present **must be original** and demonstrate your effort in solving the presented problems. **We will not tolerate** blatantly using existing solutions (such as from the internet), improper collaboration (e.g., sharing code or experimental data between groups) and plagiarism. If the honor code is not met, no points will be awarded.

 
 ## Versions
  * v1.1: Updated deadline
  * v1.0: Initial notebook
  
  ---

In [1]:
import time 
 
import numpy as np 
import matplotlib.pyplot as plt 

import json 
import time 
import pickle 
import sys 
import csv 
import os
import os.path as osp 
import shutil 
import pathlib
from pathlib import Path

from IPython.display import display, HTML
 
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots 
plt.rcParams['image.interpolation'] = 'nearest' 
plt.rcParams['image.cmap'] = 'gray' 
 
# for auto-reloading external modules 
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython 
%load_ext autoreload
%autoreload 2

In [2]:
# Some suggestions of our libraries that might be helpful for this project
from collections import Counter          # an even easier way to count
from multiprocessing import Pool         # for multiprocessing
from tqdm import tqdm                    # fancy progress bars

# Load other libraries here.
# Keep it minimal! We should be easily able to reproduce your code.

# We preload pytorch as an example
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
import gc
from sklearn.preprocessing import LabelEncoder

# Setup

  * Download the datasets: [train](https://nextcloud.mpi-klsb.mpg.de/index.php/s/pJrRGzm2So2PMZm) (128M) and [test](https://nextcloud.mpi-klsb.mpg.de/index.php/s/zN3yeWzQB3i5WqE) (92M)
  * Unpack them under `./data/train` and `./data/test`
  * Hint: you can execute shell scripts from notebooks using the `!` prefix, e.g., `! wget <url>`

In [3]:
!wget -O train.tar.gz https://nextcloud.mpi-klsb.mpg.de/index.php/s/pJrRGzm2So2PMZm/download
!wget -O test.tar.gz https://nextcloud.mpi-klsb.mpg.de/index.php/s/zN3yeWzQB3i5WqE/download

--2021-12-04 13:08:54--  https://nextcloud.mpi-klsb.mpg.de/index.php/s/pJrRGzm2So2PMZm/download
Resolving nextcloud.mpi-klsb.mpg.de (nextcloud.mpi-klsb.mpg.de)... 139.19.205.150
Connecting to nextcloud.mpi-klsb.mpg.de (nextcloud.mpi-klsb.mpg.de)|139.19.205.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134409420 (128M) [application/x-gzip]
Saving to: ‘train.tar.gz’

train.tar.gz        100%[===================>] 128.18M  19.8MB/s    in 7.5s    

2021-12-04 13:09:02 (17.1 MB/s) - ‘train.tar.gz’ saved [134409420/134409420]

--2021-12-04 13:09:02--  https://nextcloud.mpi-klsb.mpg.de/index.php/s/zN3yeWzQB3i5WqE/download
Resolving nextcloud.mpi-klsb.mpg.de (nextcloud.mpi-klsb.mpg.de)... 139.19.205.150
Connecting to nextcloud.mpi-klsb.mpg.de (nextcloud.mpi-klsb.mpg.de)|139.19.205.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96255569 (92M) [application/x-gzip]
Saving to: ‘test.tar.gz’

test.tar.gz         100%[===================

In [4]:
!mkdir data

In [5]:
!tar -xzvf train.tar.gz -C data/

Streaming output truncated to the last 5000 lines.
train/5ea1a7614b1a2d3006c30ab4f0acc050596f4366.NothingFound
train/bfc269ce4d17afbeca3711a78875ad37ad290bae.Agent
train/25c9adf3ed7b75412a32737f65740a9c7ebecf33.NothingFound
train/e67e7796e6c5b512b2e5939f4589eb231bbb6cf0.NothingFound
train/7bc19da9f54d42ee10db45d0ca02b9186b456754.NothingFound
train/ec261f42404ed8fe14d0d13df5f7ebd4748632fe.Basun
train/3936b030d69c2e05a565a5666f49e79bde79dde3.NothingFound
train/177e2b21a07483e3ae8c4992dc18d4b2281331cd.Texel
train/ba1e7893fe817375cb75b6b0b38d94bab5b229f4.AutoIt
train/38e2e93bec776f8a2882d63c97dd6fce107fe52e.Basun
train/e5569d1fe22223ddbdac7dab60c4cb51c94ddce0.Patched
train/d5ddcd54d50a3202bb4cc3d5b6fee5d9a8497017.Virut
train/6a36c3b04570ef1362509949782bcfb634ef67f5.Texel
train/dbee2d99b6dc86b70c639d14a3287553be07816e.NothingFound
train/96011308f2976258fb0e080bd636157546e94bd6.Virut
train/4fa3b55dee3780907152e5ea66e1831be33d0964.NothingFound
train/b95e242edddcf70f46cf00a07c5c8367dc142721.Te

In [6]:
!tar -xzvf test.tar.gz -C data/

Streaming output truncated to the last 5000 lines.
test/6f70d4068246bcc143f51fc47e1522fbd83f1172.NothingFound
test/cc829ee7c7c33b0310aacb2322d8b5dad16d08a6.AutoIt
test/62f4d173d3ba4236f7dc6cb66908c2af854b996f.VB
test/5f7adf5d096b7862ba0a76b3fd09bcb0ac4f2757.Texel
test/5fc403ee2a9a6aeadb9cf0bb635e58e50c3deb93.AutoIt
test/293a21dc0d71393409be8529b5efc0912860c120.Texel
test/84d27eeb086e7c8e8f39c1326645a33448d7213e.NothingFound
test/f6c369c31ba7875ae5f643c5d50e5ec90544989f.Virut
test/374e4debfa58b242d5c66197d5f95d2a20381b27.NothingFound
test/2c50ed626f776eeac61aaf830507fa75b59f2657.Texel
test/122c6e7ded47d788a6219898be9257467dc0eebc.NothingFound
test/55359d2dbe940428ea02551c1222479adbca00fc.Basun
test/44b49519e84293a28cc980ba00992f1d411bf5b9.Basun
test/dccd76778ee93b08aa31ae9c72cd0e66898a6701.Basun
test/7c0dcf67e5d6472222de0f7314cb1713403eed7d.Virut
test/74251e176a84ca089ad21045319e6ef22b6bc970.VB
test/17ab6f0b9c8e4b9f0cb0dd9b8aa489a97d49cc7e.Texel
test/8c014220325a34bb5654c931806f77f5ce88

In [7]:
# Check that you are prepared with the data
! printf '# train examples (Should be 13682) : '; ls data/train | wc -l
! printf '# test  examples (Should be 10000) : '; ls data/test | wc -l

# train examples (Should be 13682) : 13682
# test  examples (Should be 10000) : 10000


Now that you're set, let's briefly look at the data you have been handed.
Each file encodes the behavior report of a program (potentially a malware), using an encoding scheme called "The Malware Instruction Set" (MIST for short).
At this point, we highly recommend you briefly read-up Sec. 2 of the [MIST](http://www.mlsec.org/malheur/docs/mist-tr.pdf) documentation.

You will find each file named as `filename.<malwarename>`:
```
» ls data/train | head
00005ecc06ae3e489042e979717bb1455f17ac9d.NothingFound
0008e3d188483aeae0de62d8d3a1479bd63ed8c9.Basun
000d2eea77ee037b7ef99586eb2f1433991baca9.Patched
000d996fa8f3c83c1c5568687bb3883a543ec874.Basun
0010f78d3ffee61101068a0722e09a98959a5f2c.Basun
0013cd0a8febd88bfc4333e20486bd1a9816fcbf.Basun
0014aca72eb88a7f20fce5a4e000c1f7fff4958a.Texel
001ffc75f24a0ae63a7033a01b8152ba371f6154.Texel
0022d6ba67d556b931e3ab26abcd7490393703c4.Basun
0028c307a125cf0fdc97d7a1ffce118c6e560a70.Swizzor
...
```
and within each file, you will see a sequence of individual systems calls monitored duing the run-time of the binary - a malware named 'Basun' in the case:
```
» head data/train/000d996fa8f3c83c1c5568687bb3883a543ec874.Basun
# process 000006c8 0000066a 022c82f4 00000000 thread 0001 #
02 01 | 000006c8 0000066a 00015000
02 02 | 00006b2c 047c8042 000b9000
02 02 | 00006b2c 047c8042 00108000
02 02 | 00006b2c 047c8042 00153000
02 02 | 00006b2c 047c8042 00091000
02 02 | 00006b2c 047c8042 00049000
02 02 | 00006b2c 047c8042 000aa000
02 02 | 00006b2c 047c8042 00092000
02 02 | 00006b2c 047c8042 00011000
...
```
(**Note**: Please ignore the first line that begins with `# process ...`.)

Your task in this project is to train a malware detector, which given the sequence of system calls (in the MIST-formatted file like above), predicts one of 10 classes: `{ Agent, Allaple, AutoIt, Basun, NothingFound, Patched, Swizzor, Texel, VB, Virut }`, where `NothingFound` roughly represents no malware is present.
In terms of machine learning terminology, your malware detector $F: X \rightarrow Y$ should learn a mapping from the MIST-encoded behaviour report (the input $x \in X$) to the malware class $y \in Y$.

Consequently, you will primarily tackle two challenges in this project:
  1. "Vectorizing" the input data i.e., representing each input (file) as a tensor
  1. Training an ML model
  

### Some tips:
  * Begin with an extremely simple representation/ML model and get above chance-level classification performance
  * Choose your evaluation metric wisely
  * Save intermediate computations (e.g., a token to index mapping). This will avoid you parsing the entire dataset for every experiment
  * Try using `multiprocessing.Pool` to parallelize your `for` loops

---

# 1. Vectorize Data

## 1.a. Load Raw Data

In [8]:
def load_content(filepath):
    '''Given a filepath, returns (content, classname), where content = [list of lines in file]'''
    with open(filepath, "r") as file:
        data = file.read().split('\n')
    data = data[:-1]
    _, file_extension = os.path.splitext(filepath)

    return data, file_extension[1:]


def load_data(root_path, nworkers=10):
    '''Returns each data sample as a list of strings wher each string correpsonds to one file, and another list corresponding to the labels.'''
    raw_data_samples = []
    labels = []
    for file_name in tqdm(os.listdir(root_path)):
        filepath = os.path.join(root_path, file_name)
        data, label = load_content(filepath)
        lst_initials = []
        for line in data:
            if line[0]=="#":
                continue
            line = line.split(" ")
            line.remove("|")
            if len(line)>=4:
                lst_initials.extend(line[:4])
            else:
                lst_initials.extend(line[:len(line)])
        raw_data_samples.append(" ".join(lst_initials))
        labels.append(label)

    return raw_data_samples, labels

In [9]:
print('=> Loading training data ... ')
train_raw_samples, train_labels = load_data('./data/train')

=> Loading training data ... 


100%|██████████| 13682/13682 [04:39<00:00, 48.95it/s]


In [10]:
print('=> Loading testing data ... ')
test_raw_samples, test_labels = load_data('./data/test')

=> Loading testing data ... 


100%|██████████| 10000/10000 [03:25<00:00, 48.59it/s]


In [11]:
# project_mode = 'trainval'    # trainval, traintest, debug
# np.random.seed(123)          # To perform the same split across multiple runs

# if project_mode == 'trainval':
#     pass
# #     train_raw_samples, val_raw_samples = train_test_split()
# elif project_mode == 'traintest':
#     # Load train and test data
#     pass
# elif project_mode == 'debug':
#     # Optional, use a small subset of the training and validation data for fast debugging
#     pass
# else:
#     raise ValueError('Unrecognized mode')
    
# # print('=> # Train samples = ', len(train_raw_samples))
# # print('=> # Test  samples = ', len(test_raw_samples))

In [13]:
print(len(train_raw_samples), len(train_labels))
print(len(test_raw_samples), len(test_labels))

13682 13682
10000 10000


In [14]:
# Converting labels to numericals
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

In [15]:
train_raw, val_raw, labels_train, labels_val = train_test_split(train_raw_samples, train_labels, test_size=0.20, stratify=train_labels, random_state=42)
del train_raw_samples, train_labels
gc.collect()

510

In [16]:
print("Training data len: ", len(train_raw), len(labels_train))
print("Val data len: ", len(val_raw), len(labels_val))
print("Test data len: ", len(test_raw_samples), len(test_labels))

Training data len:  10945 10945
Val data len:  2737 2737
Test data len:  10000 10000


## 1.b. Vectorize: Setup

Make one pass over the inputs to identify relevant features/tokens.

Suggestion:
  - identify tokens (e.g., unigrams, bigrams)
  - create a token -> index (int) mapping. Note that you might have a >10K unique tokens. So, you will have to choose a suitable "vocabulary" size.

In [17]:
# Creating the word2index and index2word dictionaries.
def get_key_idx_map(counter, vocab_size):
    word2idx = {'_PAD': 0, '_UNK': 1}
    word2idx.update({word: i+2 for i, (word, count) in enumerate(counter.most_common(vocab_size))})
    idx2word = {idx: word for word, idx in word2idx.items()}
    return word2idx, idx2word

In [18]:
# Creating the counts of the unigrams from the training data.
counts = Counter()
for line in tqdm(train_raw):
    counts.update(line.split(" "))

100%|██████████| 10945/10945 [01:56<00:00, 93.79it/s]


In [19]:
print("Total unique words: ", len(counts))

Total unique words:  132883


In [20]:
max_vocab_size = 20000

word2idx, idx2word = get_key_idx_map(counts, max_vocab_size)

# Save vocab to file
out_path = 'application_vocab_{}.pkl'.format(max_vocab_size)
with open(out_path, 'wb') as wf:
    dct = {'word2idx': word2idx,
          'idx2word': idx2word}
    pickle.dump(dct, wf)

In [21]:
len(word2idx), len(idx2word)

(20002, 20002)

In [22]:
list(word2idx.items())[:9]

[('_PAD', 0),
 ('_UNK', 1),
 ('03', 2),
 ('02', 3),
 ('000004', 4),
 ('004000', 5),
 ('01', 6),
 ('0a', 7),
 ('047c8042', 8)]

In [23]:
list(idx2word.items())[:9]

[(0, '_PAD'),
 (1, '_UNK'),
 (2, '03'),
 (3, '02'),
 (4, '000004'),
 (5, '004000'),
 (6, '01'),
 (7, '0a'),
 (8, '047c8042')]

## 1.c. Vectorize Data

Use the (token $\rightarrow$ index) mapping you created before to vectorize your data

In [24]:
def convert_to_index(sent, max_length):
  '''Converts the word to indexed representation and also pads or truncate it to max_length'''

  sentence_indexed = []
  for token in sent.split(" "):
    if len(sentence_indexed) >= max_length:
      break
    sentence_indexed.append(word2idx.get(str(token), 1))

  length_sent = len(sentence_indexed)
  if len(sentence_indexed) < max_length:
    sentence_indexed = sentence_indexed + [0] * (max_length - len(sentence_indexed))

  return sentence_indexed, length_sent

In [25]:
def vectorize_raw_samples(raw_samples, nworkers=10):
    vectorized_samples = []
    original_lengths = []
    for line in tqdm(raw_samples):
      sentence_indexed, length = convert_to_index(line, 2000)
      vectorized_samples.append(sentence_indexed)
      original_lengths.append(length)
    return vectorized_samples, original_lengths

In [26]:
print('=> Processing: Train')
train_data, train_lengths = vectorize_raw_samples(train_raw)
del train_raw
gc.collect()

=> Processing: Train


100%|██████████| 10945/10945 [00:53<00:00, 205.16it/s]


0

In [27]:
print('=> Processing: Val')
val_data, val_lengths = vectorize_raw_samples(val_raw)
del val_raw
gc.collect()

=> Processing: Val


100%|██████████| 2737/2737 [00:13<00:00, 195.96it/s]


0

In [28]:
print('=> Processing: Test')
test_data, test_lengths = vectorize_raw_samples(test_raw_samples)
del test_raw_samples
gc.collect()

=> Processing: Test


100%|██████████| 10000/10000 [00:50<00:00, 196.63it/s]


0

In [29]:
train_data, labels_train, train_lengths = torch.LongTensor(train_data), torch.LongTensor(labels_train), torch.Tensor(train_lengths)
val_data, labels_val, val_lengths = torch.LongTensor(val_data), torch.LongTensor(labels_val), torch.Tensor(val_lengths)
test_data, labels_test, test_lengths = torch.LongTensor(test_data), torch.LongTensor(test_labels), torch.Tensor(test_lengths)

In [30]:
print(train_data.shape, labels_train.shape, train_lengths.shape)
print(val_data.shape, labels_val.shape, val_lengths.shape)
print(test_data.shape, labels_test.shape, test_lengths.shape)

torch.Size([10945, 2000]) torch.Size([10945]) torch.Size([10945])
torch.Size([2737, 2000]) torch.Size([2737]) torch.Size([2737])
torch.Size([10000, 2000]) torch.Size([10000]) torch.Size([10000])


In [31]:
# Vocab size to create the embedding layer
vocab_size = len(word2idx)

trainset = TensorDataset(train_data, labels_train, train_lengths)
validset = TensorDataset(val_data, labels_val, val_lengths)
testset = TensorDataset(test_data, labels_test, test_lengths)

train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(validset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [32]:
# Example of using dataloader
for batch_idx, (sents, labels, lens) in enumerate(train_loader):
  print(sents.shape, labels.shape, lens.shape)
  break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([64, 2000]) torch.Size([64]) torch.Size([64])


# 2. Train Model

You will now train an ML model on the vectorized datasets you created previously.

_Note_: Although we often refer to each input as a 'vector' for simplicity, each of your inputs can also be higher dimensional tensors.

## 2.a. Helpers

In [ ]:
# Feel free to edit anything in this block

def evaluate_preds(y_gt, y_pred):
    pass


def another_helper(question):
    return 42


def save_model(model, out_path):
    pass


def save_data(eval_data, out_path):
    with open(out_path, 'wb') as wf:
        pickle.dump(eval_data, out_path)

## 2.b. Define Model

Describe your model here.

In [ ]:
# Feel free to edit anything in this block

class NetTemplate(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NetTemplate, self).__init__()
        # Layer definitions

    def forward(self, x):
        # Forward pass
        return x

## 2.c. Set Hyperparameters

In [ ]:
# Define your hyperparameters here

in_dims = trainset[0][0].shape[0]
out_dims = len(class_to_idx)
...

# Optimization
n_epochs = ...
batch_size = ...
lr = ...
...

## 2.d. Train your Model

In [ ]:
# Feel free to edit anything in this block

# model = SomeModel(...)
# model.train()

# Optimizer
# optimizer = None

# Data Loaders
# trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
# testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# Etc.

In [ ]:
#
#
# ------- Your Code -------
#
# Example:
# for epoch in range(n_epochs):
#     ... train ...
#     ... validate ...

## 2.e. Evaluate model

In [ ]:
#
#
# ------- Your Code -------
#
# 

## 2.f. Save Model + Data

In [ ]:
#
#
# ------- Your Code -------
#
# 

---

# 3. Analysis

## 3.a. Summary: Main Results

Summarize your approach and results here

## 3.b. Discussion

Enter your final summary here.

For instance, you can address:
- What was the performance you obtained with the simplest approach?
- Which vectorized input representations helped more than the others?
- Which malwares are difficult to detect and why?
- Which approach do you recommend to perform malware classification?